<h1>Land Degradation - Land Cover Change</h1>

Porting of the code from Trends.Earth: https://github.com/ConservationInternational/landdegradation/blob/master/landdegradation/land_cover.py

First, check if the IPython Widgets library is available on the server.

In [1]:
# Code to check the IPython Widgets library.
try:
  import ipywidgets
  print('The IPython Widgets library (version {0}) is available on this server.'.format(
      ipywidgets.__version__
    ))
except ImportError:
  print('The IPython Widgets library is not available on this server.\n'
        'Please see https://github.com/jupyter-widgets/ipywidgets '
        'for information on installing the library.')
  raise

The IPython Widgets library (version 7.5.0) is available on this server.


Next, check if the Earth Engine API is available on the server.

In [2]:
# Code to check the Earth Engine API library.
try:
  import ee
  print('The Earth Engine Python API (version {0}) is available on this server.'.format(
      ee.__version__
    ))
except ImportError:
  print('The Earth Engine Python API library is not available on this server.\n'
        'Please see https://developers.google.com/earth-engine/python_install '
        'for information on installing the library.')
  raise

The Earth Engine Python API (version 0.1.185) is available on this server.


Finally, check if the notebook server is authorized to access the Earth Engine backend servers.

In [3]:
#Define the TEImage class
class TEImage(object):
    "A class to store GEE images and band info for export to cloud storage"
    def __init__(self, image, band_info):
        self.image = image
        self.band_info = band_info

        self._check_validity()
    
    def _check_validity(self):
        if len(self.band_info) != len(self.image.getInfo()['bands']):
            raise GEEImageError('Band info length ({}) does not match number of bands in image ({})'.format(len(self.band_info),
                                                                                                            len(self.image.getInfo()['bands'])))

    def merge(self, other):
        "Merge with another TEImage object"
        self.image = self.image.addBands(other.image)
        self.band_info.extend(other.band_info)

        self._check_validity()

    def addBands(self, bands, band_info):
        "Add new bands to the image"
        self.image = self.image.addBands(bands)
        self.band_info.extend(band_info)

        self._check_validity()

    def selectBands(self, band_names):
        "Select certain bands from the image, dropping all others"
        band_indices = [i for i, bi in enumerate(self.band_info) if bi.name in band_names]
        if len(band_indices) < 1:
            raise GEEImageError('Bands "{}" not in image'.format(band_names))

        self.band_info = [self.band_info[i] for i in band_indices]
        self.image = self.image.select(band_indices)

        self._check_validity()

    def setAddToMap(self, band_names=[]):
        "Set the layers that will be added to the user's map in QGIS by default"
        for i in range(len(self.band_info)):
            if self.band_info[i].name in band_names:
                self.band_info[i].add_to_map = True
            else:
                self.band_info[i].add_to_map = False

    def export(self, geojsons, task_name, crs, logger, execution_id=None, 
               proj=None):
        "Export layers to cloud storage"
        if not execution_id:
            execution_id = str(random.randint(1000000, 99999999))
        else:
            execution_id = execution_id

        if not proj:
            proj = self.image.projection()

        tasks = []
        n = 1
        for geojson in geojsons:
            if task_name:
                out_name = '{}_{}_{}'.format(execution_id, task_name, n)
            else:
                out_name = '{}_{}'.format(execution_id, n)

            export = {'image': self.image,
                      'description': out_name,
                      'fileNamePrefix': out_name,
                      'bucket': BUCKET,
                      'maxPixels': 1e13,
                      'crs': crs,
                      'scale': ee.Number(proj.nominalScale()).getInfo(),
                      'region': get_coords(geojson)}
            t = gee_task(ee.batch.Export.image.toCloudStorage(**export),
                         out_name, logger)
            tasks.append(t)
            n+=1
            
        logger.debug("Exporting to cloud storage.")
        urls = []
        for task in tasks:
            task.join()
            urls.extend(task.get_urls())

        gee_results = CloudResults(task_name,
                                   self.band_info,
                                   urls)
        results_schema = CloudResultsSchema()
        json_results = results_schema.dump(gee_results)

        return json_results

In [4]:
# Schema for storing information on bands
class BandInfo(object):
    def __init__(self, name, add_to_map=False, activated=True, metadata={}, 
                 no_data_value=-32768):
        self.name = name
        self.no_data_value = no_data_value
        self.add_to_map = add_to_map
        self.activated = activated
        self.metadata = metadata

In [5]:
# Code to check if authorized to access Earth Engine.
import cStringIO
import os
import urllib

def isAuthorized():
  try:
    ee.Initialize()
    return True
  except:
    return False

form_item_layout = ipywidgets.Layout(width="100%", align_items='center')
  
if isAuthorized():
  
  def revoke_credentials(sender):
    credentials = ee.oauth.get_credentials_path()
    if os.path.exists(credentials):
      os.remove(credentials)
    print('Credentials have been revoked.')
  
  # Define widgets that may be displayed.
  auth_status_button = ipywidgets.Button(
    layout=form_item_layout,
    disabled = True,
    description = 'The server is authorized to access Earth Engine',
    button_style = 'success',
    icon = 'check'
  )
  
  instructions = ipywidgets.Button(
    layout=form_item_layout,
    description = 'Click here to revoke authorization',
    button_style = 'danger',
    disabled = False,
  )
  instructions.on_click(revoke_credentials)

else:
  
  def save_credentials(sender):
    try:
      token = ee.oauth.request_token(get_auth_textbox.value.strip())
    except Exception as e:
      print(e)
      return
    ee.oauth.write_token(token)
    get_auth_textbox.value = ''  # Clear the textbox.
    print('Successfully saved authorization token.')

  # Define widgets that may be displayed.
  get_auth_textbox = ipywidgets.Text(
    placeholder='Paste authorization code here',
    description='Authentication Code:'
  )
  get_auth_textbox.on_submit(save_credentials)

  auth_status_button = ipywidgets.Button(
    layout=form_item_layout,
    button_style = 'danger',
    description = 'The server is not authorized to access Earth Engine',
    disabled = True
  )
  
  instructions = ipywidgets.VBox(
    [
      ipywidgets.HTML(
        'Click on the link below to start the authentication and authorization process. '
        'Once you have received an authorization code, paste it in the box below and press return.'
      ),
      ipywidgets.HTML(
        '<a href="{url}" target="auth">Open Authentication Tab</a><br/>'.format(
          url=ee.oauth.get_authorization_url()
        )
      ),
      get_auth_textbox
    ],
    layout=form_item_layout
  )

# Display the form.
form = ipywidgets.VBox([
  auth_status_button,
  instructions
])
form

VkJveChjaGlsZHJlbj0oQnV0dG9uKGJ1dHRvbl9zdHlsZT11J3N1Y2Nlc3MnLCBkZXNjcmlwdGlvbj11J1RoZSBzZXJ2ZXIgaXMgYXV0aG9yaXplZCB0byBhY2Nlc3MgRWFydGggRW5naW5lJyzigKY=


Once the server is authorized, you can retrieve data from Earth Engine and use it in the notebook.

In [6]:
#initalize necessary variables
year_baseline = 2001
year_target = 2015
trans_matrix = [0, -1, -1, -1, -1, -1, 0, 
                1, 0, 1, -1, -1, -1, 0, 
                1, -1, 0, -1, -1, -1, 0, 
                -1, -1, -1, 0, -1, -1, 0, 
                1, 1, 1, 1, 0, 1, 0, 
                1, 1, 1, 1, -1, 0, 0, 
                0, 0, 0, 0, 0, 0, 0]
remap_matrix = [[10, 11, 12, 20, 30, 40, 50, 
                60, 61, 62, 70, 71, 72, 80, 
                81, 82, 90, 100, 110, 120, 121, 
                122, 130, 140, 150, 151, 152, 153, 
                160, 170, 180, 190, 200, 201, 202, 
                210, 220], 
                [3, 3, 3, 3, 3, 2, 1, 
                1, 1, 1, 1, 1, 1, 1, 
                1, 1, 1, 1, 2, 2, 2, 
                2, 2, 2, 2, 2, 2, 2, 
                4, 4, 4, 5, 6, 6, 6, 
                7, 6]]

In [7]:
# Code to display an Earth Engine generated image.
from IPython.display import Image

#land cover
lc = ee.Image("users/geflanddegradation/toolbox_datasets/lcov_esacc_1992_2015")
lc = lc.where(lc.eq(9999), -32768)
lc = lc.updateMask(lc.neq(-32768))

In [8]:
# Remap LC according to input matrix
lc_remapped = lc.select('y{}'.format(year_baseline)).remap(remap_matrix[0], remap_matrix[1])
for year in range(year_baseline + 1, year_target + 1):
    lc_remapped = lc_remapped.addBands(lc.select('y{}'.format(year)).remap(remap_matrix[0], remap_matrix[1]))

In [9]:
## target land cover map reclassified to IPCC 6 classes
lc_bl = lc_remapped.select(0)

## baseline land cover map reclassified to IPCC 6 classes
lc_tg = lc_remapped.select(len(lc_remapped.getInfo()['bands']) - 1)

## compute transition map (first digit for baseline land cover, and second digit for target year land cover)
lc_tr = lc_bl.multiply(10).add(lc_tg)

In [10]:
## definition of land cover transitions as degradation (-1), improvement (1), or no relevant change (0)
lc_dg = lc_tr.remap([11, 12, 13, 14, 15, 16, 17,
                         21, 22, 23, 24, 25, 26, 27,
                         31, 32, 33, 34, 35, 36, 37,
                         41, 42, 43, 44, 45, 46, 47,
                         51, 52, 53, 54, 55, 56, 57,
                         61, 62, 63, 64, 65, 66, 67,
                         71, 72, 73, 74, 75, 76, 77],
                        trans_matrix)

In [11]:
## Remap persistence classes so they are sequential. This
## makes it easier to assign a clear color ramp in QGIS.
lc_tr = lc_tr.remap([11, 12, 13, 14, 15, 16, 17,
                         21, 22, 23, 24, 25, 26, 27,
                         31, 32, 33, 34, 35, 36, 37,
                         41, 42, 43, 44, 45, 46, 47,
                         51, 52, 53, 54, 55, 56, 57,
                         61, 62, 63, 64, 65, 66, 67,
                         71, 72, 73, 74, 75, 76, 77],
                        [1, 12, 13, 14, 15, 16, 17,
                         21, 2, 23, 24, 25, 26, 27,
                         31, 32, 3, 34, 35, 36, 37,
                         41, 42, 43, 4, 45, 46, 47,
                         51, 52, 53, 54, 5, 56, 57,
                         61, 62, 63, 64, 65, 6, 67,
                         71, 72, 73, 74, 75, 76, 7])

In [12]:
out = TEImage(lc_dg.addBands(lc.select('y{}'.format(year_baseline))).addBands(lc.select('y{}'.format(year_target))).addBands(lc_tr),
                  [BandInfo("Land cover (degradation)", add_to_map=True, metadata={'year_baseline': year_baseline, 'year_target': year_target}),
                   BandInfo("Land cover (ESA classes)", metadata={'year': year_baseline}),
                   BandInfo("Land cover (ESA classes)", metadata={'year': year_target}),
                   BandInfo("Land cover transitions", add_to_map=True, metadata={'year_baseline': year_baseline, 'year_target': year_target})])

In [13]:
d_lc = []
for year in range(year_baseline, year_target + 1):
    if (year == year_baseline) or (year == year_target):
         add_to_map = True
    else:
        add_to_map = False
    d_lc.append(BandInfo("Land cover (7 class)", add_to_map=add_to_map, metadata={'year': year}))
out.addBands(lc_remapped, d_lc)

out.image = out.image.unmask(-32768).int16()
from IPython.display import Image
url = ee.Image(out.image).getThumbUrl({'min':0, 'max':3000})
Image(url=url)